# PlantoActual

## 1. Read data from MySQL
### 1.1. Import important packages

In [1]:
import pymysql
import pandas as pd
import decimal

### 1.2. Connect to MySQL database reconai

In [2]:
mydb = pymysql.connect(host='localhost',
                             user='root',
                             password='peacebro',database='reconai')
print(mydb)

### 1.3. Read data from database

In [3]:
sql_merged_sales='SELECT * FROM `merged_sales`;'
sql_plandata='SELECT * FROM `plandata`'

In [4]:
df_merged = pd.read_sql(sql_merged_sales, mydb)
df_merged.head()

,CPG,CPG Region,Channel,Retailer,Retailer Region,Year,Week,Category,Brand,UPC,PPG,Sales
0,ABC Corp,North,Mass,SuperMart,North,2017,1,Widgets,A,1234567890,PPG1,"5,83,780.00"
1,ABC Corp,East,Mass,SuperMart,East,2017,1,Widgets,A,1234567890,PPG1,"6,59,530.00"
2,ABC Corp,South,Mass,SuperMart,Gulf Coast,2017,1,Widgets,A,1234567890,PPG1,"4,80,659.00"
3,ABC Corp,South,Mass,SuperMart,Deep South,2017,1,Widgets,A,1234567890,PPG1,"3,32,290.00"
4,ABC Corp,West,Mass,SuperMart,West,2017,1,Widgets,A,1234567890,PPG1,"6,64,378.00"


In [5]:
df_plandata = pd.read_sql(sql_plandata, mydb)
df_plandata.head()

,CPG,CPG Region,Channel,Retailer,Retailer Region,Year,Week,Category,Brand,UPC,PPG,Sales
0,ABC Corp,North,Mass,SuperMart,North,2017,1,Widgets,A,1234567890,PPG1,"5,78,000.00"
1,ABC Corp,East,Mass,SuperMart,East,2017,1,Widgets,A,1234567890,PPG1,"6,53,000.00"
2,ABC Corp,South,Mass,SuperMart,Gulf Coast,2017,1,Widgets,A,1234567890,PPG1,"4,75,900.00"
3,ABC Corp,South,Mass,SuperMart,Deep South,2017,1,Widgets,A,1234567890,PPG1,"3,29,000.00"
4,ABC Corp,West,Mass,SuperMart,West,2017,1,Widgets,A,1234567890,PPG1,"6,57,800.00"


## 2. Editing Dataframe for merge
### 2.1. Convert Sales column to float type

In [6]:
def sales_product(df):
    #df['date'] = pd.to_datetime(df.assign(day=1, month=1)[['Year', 'month', 'day']])+pd.to_timedelta(df.Week*7, unit='days')
    df['Sales'] = df['Sales'].str.replace(',', '')
    df['Sales'] = df['Sales'].astype(float)
sales_product(df_plandata)
sales_product(df_merged)

### 2.2. Summing Gulf Coast and Deep South rows together

In [7]:
def merge_south(df):
    for year in [2017,2018]:
        for week in range(1,53):
            for UPC in [1234567890,1234512345]:
                ex=df[(df['Retailer']=='SuperMart') & (df['CPG Region']=='South') & 
                    (df['Year']==year) & (df['Week']==week) & 
                    (df['UPC']==1234567890)].sum()
                ex['Sales']=round(ex['Sales'], 2)
                df.loc[(df['Retailer']=='SuperMart') & (df['CPG Region']=='South') & 
                    (df['Year']==year) & (df['Week']==week) & 
                    (df['UPC']==UPC)&(df['Category']=='Widgets'),
                    'Sales'] = ex['Sales']
    return df
df_plandata=merge_south(df_plandata)
df_plandata.drop_duplicates(subset =["UPC","CPG Region","Retailer","Sales","Year","Week"], 
                     keep = "first", inplace = True)
df_merged=merge_south(df_merged)
df_merged.drop_duplicates(subset =["UPC","CPG Region","Retailer","Sales","Year","Week"], 
                     keep = "first", inplace = True)

In [8]:
df_plandata.head()

,CPG,CPG Region,Channel,Retailer,Retailer Region,Year,Week,Category,Brand,UPC,PPG,Sales
0,ABC Corp,North,Mass,SuperMart,North,2017,1,Widgets,A,1234567890,PPG1,578000.0
1,ABC Corp,East,Mass,SuperMart,East,2017,1,Widgets,A,1234567890,PPG1,653000.0
2,ABC Corp,South,Mass,SuperMart,Gulf Coast,2017,1,Widgets,A,1234567890,PPG1,804900.0
4,ABC Corp,West,Mass,SuperMart,West,2017,1,Widgets,A,1234567890,PPG1,657800.0
5,ABC Corp,North,Mass,BigCo,North,2017,1,Widgets,A,1234567890,PPG1,198400.0


## 3. Merge dataframes
### 3.1. Rename Sales column in both the dataframes

In [9]:
df_merged=df_merged.rename(columns={'Sales': 'Actual numbers'})

In [10]:
df_plandata=df_plandata.rename(columns={'Sales': 'Plan numbers'})
df_plandata.head()

,CPG,CPG Region,Channel,Retailer,Retailer Region,Year,Week,Category,Brand,UPC,PPG,Plan numbers
0,ABC Corp,North,Mass,SuperMart,North,2017,1,Widgets,A,1234567890,PPG1,578000.0
1,ABC Corp,East,Mass,SuperMart,East,2017,1,Widgets,A,1234567890,PPG1,653000.0
2,ABC Corp,South,Mass,SuperMart,Gulf Coast,2017,1,Widgets,A,1234567890,PPG1,804900.0
4,ABC Corp,West,Mass,SuperMart,West,2017,1,Widgets,A,1234567890,PPG1,657800.0
5,ABC Corp,North,Mass,BigCo,North,2017,1,Widgets,A,1234567890,PPG1,198400.0


### 3.2. Reset the index of the dataframes for merge

In [11]:
df_plandata=df_plandata.reset_index(drop=True)
df_merged=df_merged.reset_index(drop=True)
df_plandata.head()

,CPG,CPG Region,Channel,Retailer,Retailer Region,Year,Week,Category,Brand,UPC,PPG,Plan numbers
0,ABC Corp,North,Mass,SuperMart,North,2017,1,Widgets,A,1234567890,PPG1,578000.0
1,ABC Corp,East,Mass,SuperMart,East,2017,1,Widgets,A,1234567890,PPG1,653000.0
2,ABC Corp,South,Mass,SuperMart,Gulf Coast,2017,1,Widgets,A,1234567890,PPG1,804900.0
3,ABC Corp,West,Mass,SuperMart,West,2017,1,Widgets,A,1234567890,PPG1,657800.0
4,ABC Corp,North,Mass,BigCo,North,2017,1,Widgets,A,1234567890,PPG1,198400.0


### 3.3. Sales conversion
#### 3.3.1. Function to convert value to indian currency

In [12]:
def currency_in_indian_format(n):
    """ Convert a number (int / float) into indian formatting style """
    d = decimal.Decimal(str(n))

    if d.as_tuple().exponent < -2:
        s = str(n)
    else:
        s = '{0:.2f}'.format(n)

    l = len(s)
    i = l - 1

    res, flag, k = '', 0, 0
    while i >= 0:
        if flag == 0:
            res += s[i]
            if s[i] == '.':
                flag = 1
        elif flag == 1:
            k += 1
            res += s[i]
            if k == 3 and i - 1 >= 0:
                res += ','
                flag = 2
                k = 0
        else:
            k += 1
            res += s[i]
            if k == 2 and i - 1 >= 0:
                res += ','
                flag = 2
                k = 0
        i -= 1

    return res[::-1]

#### 3.3.2. Converting Plan numbers column to indian currency

In [14]:
for i in range (0,len(df_plandata.index)):
    a=df_plandata['Plan numbers'][i]
    b=currency_in_indian_format(a)
    df_plandata['Plan numbers'][i]=b

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


#### 3.3.2. Converting Actual numbers column to indian currency

In [15]:
for i in range (0,len(df_merged.index)):
    a=df_merged['Actual numbers'][i]
    b=currency_in_indian_format(a)
    df_merged['Actual numbers'][i]=b

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### 3.4. Merge both dataframes

In [16]:
df_plantoactual_trial=pd.merge(df_plandata, df_merged, on=("Retailer Region","CPG","CPG Region","Channel","Year","Week",
                                   "Category","Brand","UPC","PPG","Retailer"))
df_plantoactual_trial.head()

,CPG,CPG Region,Channel,Retailer,Retailer Region,Year,Week,Category,Brand,UPC,PPG,Plan numbers,Actual numbers
0,ABC Corp,North,Mass,SuperMart,North,2017,1,Widgets,A,1234567890,PPG1,"5,78,000.00","5,83,780.00"
1,ABC Corp,East,Mass,SuperMart,East,2017,1,Widgets,A,1234567890,PPG1,"6,53,000.00","6,59,530.00"
2,ABC Corp,South,Mass,SuperMart,Gulf Coast,2017,1,Widgets,A,1234567890,PPG1,"8,04,900.00","8,12,949.00"
3,ABC Corp,West,Mass,SuperMart,West,2017,1,Widgets,A,1234567890,PPG1,"6,57,800.00","6,64,378.00"
4,ABC Corp,North,Mass,BigCo,North,2017,1,Widgets,A,1234567890,PPG1,"1,98,400.00","2,00,384.00"


In [17]:
df_plantoactual=pd.DataFrame(df_plantoactual_trial)
df_plantoactual=df_plantoactual.drop(['CPG','PPG','Channel','Retailer Region'],axis=1)
df_plantoactual.head()

,CPG Region,Retailer,Year,Week,Category,Brand,UPC,Plan numbers,Actual numbers
0,North,SuperMart,2017,1,Widgets,A,1234567890,"5,78,000.00","5,83,780.00"
1,East,SuperMart,2017,1,Widgets,A,1234567890,"6,53,000.00","6,59,530.00"
2,South,SuperMart,2017,1,Widgets,A,1234567890,"8,04,900.00","8,12,949.00"
3,West,SuperMart,2017,1,Widgets,A,1234567890,"6,57,800.00","6,64,378.00"
4,North,BigCo,2017,1,Widgets,A,1234567890,"1,98,400.00","2,00,384.00"


### 4. Export as csv

In [18]:
df_plantoactual.to_csv("plantoactual.csv",index = False)